In [28]:
from openai import OpenAI

client = OpenAI(
    # This is the default and can be omitted
    api_key="dummy-key",
    base_url="http://localhost:9000/v2/models/chatgpt/infer"
)

completion = client.chat.completions.create(
    model="gpt-4.1",
    messages=[
      {
        "role": "user",
        "content": "You are a helpful assistant."
      },
      {
      	"role": "assistant",
      	"content": "Hello! How can I help you?"
      },
      {
      	"role": "user",
      	"content": "What is the capital of Romania?"
      }
    ],
)

In [29]:
completion.choices[0].message.content

'The capital of Romania is Bucharest.'

## Function calling

In [31]:
from openai import OpenAI

client = OpenAI(
    # This is the default and can be omitted
    api_key="dummy-key",
    base_url="http://localhost:9000/v2/models/chatgpt/infer"
)

tools = [{
    "type": "function",
    "function": {
        "name": "get_weather",
        "description": "Get current temperature for a given location.",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "City and country e.g. Bogotá, Colombia"
                }
            },
            "required": [
                "location"
            ],
            "additionalProperties": False
        },
        "strict": True
    }
}]

completion = client.chat.completions.create(
    model="gpt-4.1",
    messages=[{"role": "system", "content": "You are a helpful assitant"}, {"role": "user", "content": "What is the weather like in Paris today?"}],
    tools=tools
)

print(completion.choices[0].message.tool_calls)

[ChatCompletionMessageToolCall(id='call_8AJuPCObHHkXWB0HIE3A0ocl', function=Function(arguments='{"location":"Paris, France"}', name='get_weather'), type='function')]


In [32]:
import requests

def get_weather(latitude, longitude):
    response = requests.get(f"https://api.open-meteo.com/v1/forecast?latitude={latitude}&longitude={longitude}&current=temperature_2m,wind_speed_10m&hourly=temperature_2m,relative_humidity_2m,wind_speed_10m")
    data = response.json()
    return data['current']['temperature_2m']

In [34]:
from openai import OpenAI
import json

client = OpenAI(
    # This is the default and can be omitted
    api_key="dummy-key",
    base_url="http://localhost:9000/v2/models/chatgpt/infer"
)

tools = [{
    "type": "function",
    "function": {
        "name": "get_weather",
        "description": "Get current temperature for provided coordinates in celsius.",
        "parameters": {
            "type": "object",
            "properties": {
                "latitude": {"type": "number"},
                "longitude": {"type": "number"}
            },
            "required": ["latitude", "longitude"],
            "additionalProperties": False
        },
        "strict": True
    }
}]

messages = [{"role": "system", "content": "You are a helpful assitant"}, {"role": "user", "content": "What's the weather like in Paris today?"}]

completion = client.chat.completions.create(
    model="gpt-4.1",
    messages=messages,
    tools=tools,
)

In [35]:
completion.choices[0].message.tool_calls

[ChatCompletionMessageToolCall(id='call_Xb3u71OXjTYykw5kBcApO2fg', function=Function(arguments='{"latitude":48.8566,"longitude":2.3522}', name='get_weather'), type='function')]

In [36]:
tool_call = completion.choices[0].message.tool_calls[0]
args = json.loads(tool_call.function.arguments)

result = get_weather(args["latitude"], args["longitude"])

In [37]:
messages.append(completion.choices[0].message)  # append model's function call message
messages.append({                               # append result message
    "role": "tool",
    "tool_call_id": tool_call.id,
    "content": str(result)
})

completion_2 = client.chat.completions.create(
    model="gpt-4.1",
    messages=messages,
    tools=tools,
)

In [38]:
completion_2.choices[0].message.content

'The current temperature in Paris is 20.2°C.'

In [39]:
messages.append({"role": "user", "content": "What's the weather like in London today?"})

In [40]:
completion_3 = client.chat.completions.create(
    model="gpt-4.1",
    messages=messages,
    tools=tools,
)

In [41]:
completion_3

ChatCompletion(id='chatcmpl-BtudQmjw4QzNH1aEYxFSxz4LyELOC', choices=[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_e1IzSSZV62vJUFCAJRhlfj2z', function=Function(arguments='{"latitude":51.5074,"longitude":-0.1278}', name='get_weather'), type='function')]))], created=1752665444, model='gpt-3.5-turbo-0125', object='chat.completion', service_tier='default', system_fingerprint=None, usage=CompletionUsage(completion_tokens=24, prompt_tokens=116, total_tokens=140, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))

In [42]:
tool_call = completion_3.choices[0].message.tool_calls[0]
args = json.loads(tool_call.function.arguments)

result = get_weather(args["latitude"], args["longitude"])

In [43]:
result

22.0

In [44]:
messages.append(completion_3.choices[0].message)  # append model's function call message
messages.append({                               # append result message
    "role": "tool",
    "tool_call_id": tool_call.id,
    "content": str(result)
})

completion_4 = client.chat.completions.create(
    model="gpt-4.1",
    messages=messages,
    tools=tools,
)

In [45]:
completion_4.choices[0].message.content

'The current weather in Paris is 20.2°C and in London is 22.0°C today.'

In [20]:
messages

[{'role': 'user', 'content': "What's the weather like in Paris today?"},
 ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_mtf3Z9rU8cj7oFGSsjAtko4R', function=Function(arguments='{"latitude":48.8566,"longitude":2.3522}', name='get_weather'), type='function')]),
 {'role': 'tool',
  'tool_call_id': 'call_mtf3Z9rU8cj7oFGSsjAtko4R',
  'content': '19.1'},
 {'role': 'user', 'content': "What's the weather like in London today?"},
 ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_KY80OAoOKphGlZxKIj21KvRy', function=Function(arguments='{"latitude":51.5074,"longitude":-0.1278}', name='get_weather'), type='function')]),
 {'role': 'tool',
  'tool_call_id': 'call_KY80OAoOKphGlZxKIj21KvRy',
  'content': '20.5'}]